# Options Configuration: Tuning Your Quantum Jobs (EXAM TESTED!)

> **Exam Weight**: Part of 15% (Section 4) | **Must Master**: ✅✅

## Learning Objectives
By the end of this notebook, you will be able to:
- Configure Options objects for Sampler and Estimator primitives
- Set execution options (shots, init_qubits)
- Understand optimization_level (0-3) tradeoffs
- Configure resilience_level for error mitigation
- Remember the default shots value (4096, not 1024!)

---

## 🧠 Conceptual Deep Dive

### The Camera Settings Analogy 📷

Think of **Options** as the **settings menu** on a professional camera before taking a photo:

| Camera Setting | Options Equivalent | Purpose |
|----------------|-------------------|---------|
| **ISO** | `optimization_level` | Quality vs speed tradeoff |
| **Flash** | `resilience_level` | Compensate for poor conditions (noise) |
| **Burst Mode** | `execution.shots` | How many photos to average |
| **Stabilization** | `dynamical_decoupling` | Fight motion blur (decoherence) |

```
Manual Mode (Options) vs Auto Mode (Defaults):
┌─────────────────────────────────────────────────────────────┐
│                                                             │
│  🔧 Auto Mode (Defaults):      📸 Manual Mode (Custom):    │
│  ┌───────────────────┐         ┌───────────────────┐       │
│  │ optimization = 2  │         │ optimization = 3  │       │
│  │ resilience = 0    │   →     │ resilience = 1    │       │
│  │ shots = 4096      │         │ shots = 8192      │       │
│  └───────────────────┘         └───────────────────┘       │
│           ↓                            ↓                    │
│    Generic results              Optimized results           │
│    ~50% accuracy                ~85% accuracy               │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Options Hierarchy (EXAM CRITICAL!)
```
Options()
├── optimization_level (0-3)    ← Circuit compilation
├── execution
│   ├── shots (default: 4096)   ← ⚠️ NOT 1024!
│   └── init_qubits             ← Reset qubits
├── resilience
│   └── resilience_level (0-2)  ← Error mitigation
└── simulator
    └── seed_simulator          ← Reproducibility
```

> **Memory Trick**: "Options = OERs" - **O**ptimization, **E**xecution, **R**esilience, **S**imulator

---

## Setup

In [ ]:
# Options Configuration Demo
from qiskit_ibm_runtime import Options

options = Options()

print("Options Configuration:")
print(f"  optimization_level: {options.optimization_level}")
print(f"  resilience_level: {options.resilience_level}")
print(f"  execution.shots: {options.execution.shots}")

print("\nNested namespaces available:")
print("  - options.execution.*    (shots, init_qubits)")
print("  - options.simulator.*    (seed_simulator)")
print("  - options.resilience.*   (noise_factors, etc.)")
print("  - options.optimization_level (0-3)")
print("  - options.resilience_level (0-2)")

print("\n✓ Imported from qiskit_ibm_runtime")
print("✓ Passed to Sampler/Estimator constructor")
print("✓ Can update after creation")

## Part 1: Options Basics

### ⚠️ EXAM TRAP: Default Shots Value

```python
# Default for Options
options.execution.shots = 4096  # ✓ Default

# NOT 1024 (old default)
# NOT 100 or 1000
```

**CRITICAL**: Default is **4096**, not 1024!

## Part 2: Execution Options

In [ ]:
# Execution Options Demo
from qiskit_ibm_runtime import Options

options = Options()

# Show defaults
print("Default Execution Options:")
print(f"  shots: {options.execution.shots}")
print(f"  init_qubits: {options.execution.init_qubits}")

# Set custom values
options.execution.shots = 1000
options.execution.init_qubits = True

print("\nAfter modification:")
print(f"  shots: {options.execution.shots}")
print(f"  init_qubits: {options.execution.init_qubits}")

print("\n🎯 EXAM TIP: Access via options.execution.shots NOT options.shots!")

## Part 3: Optimization Level

In [ ]:
# Optimization and Resilience Levels Demo
from qiskit_ibm_runtime import Options

options = Options()

print("Optimization Level (default):", options.optimization_level)
print("Resilience Level (default):", options.resilience_level)

print("\n--- Optimization Levels ---")
print("  0 - No optimization")
print("  1 - Light optimization (transpile default)")
print("  2 - Moderate (Options default) ✓")
print("  3 - Heavy optimization")

# Set custom optimization level
options.optimization_level = 3
print(f"\nAfter setting to 3: {options.optimization_level}")

print("\n⚠️ CRITICAL DIFFERENCE:")
print("  transpile() default: optimization_level=1")
print("  Options() default:   optimization_level=2")

print("\n--- Resilience Levels ---")
print("  0 - No error mitigation (default)")
print("  1 - TREX (Twirled Readout Error Extinction)")
print("  2 - ZNE (Zero-Noise Extrapolation)")

# Set resilience level
options.resilience_level = 1
print(f"\nAfter setting resilience to 1: {options.resilience_level}")

print("\n✓ Higher level = Better accuracy")
print("✓ Higher level = More shots/overhead")
print("✓ Start with level 1 for real hardware")

### ⚠️ EXAM TRAP: Resilience Levels

```python
# CORRECT - resilience_level (singular)
options.resilience_level = 1  # ✓

# WRONG - resilience.level (nested)
options.resilience.level = 1  # ❌
```

**Note**: Top-level attribute, NOT nested like execution.shots!

### Visual: Error Mitigation Effect

```
Without Mitigation (Level 0):       With M3 (Level 1):
    
Ideal: 50% |00⟩, 50% |11⟩         Ideal: 50% |00⟩, 50% |11⟩

Measured:                          Corrected:
|00⟩: ████████ 45%                |00⟩: █████████ 49%
|01⟩: █ 3%   (error!)             |01⟩:  1%
|10⟩: █ 2%   (error!)             |10⟩:  <1%
|11⟩: ████████ 50%                |11⟩: ██████████ 50%

Readout errors visible!            Corrected to near-ideal!
```

### Resilience Level Guide

| Level | Method | Overhead | Use Case |
|-------|--------|----------|----------|
| 0 | None | 0% | Simulator, debugging |
| 1 | M3 (Matrix-free Measurement Mitigation) | ~20% | Most production |
| 2 | ZNE + M3 (Zero-Noise Extrapolation) | ~300% | Critical accuracy |

## Part 5: Simulator Options

In [ ]:
# Simulator Options Demo
from qiskit_ibm_runtime import Options

options = Options()

# Set seed for reproducibility
options.simulator.seed_simulator = 42

print("Simulator Options:")
print(f"  seed_simulator: {options.simulator.seed_simulator}")

print("\n✓ seed_simulator ensures reproducible results")
print("✓ Useful for testing and debugging")
print("✓ Ignored by real backends")

## Part 6: Complete Configuration Pattern

In [ ]:
# Complete Options Configuration Pattern
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit_ibm_runtime import Options
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Create and configure options
options = Options()
options.optimization_level = 3
options.resilience_level = 1
options.execution.shots = 2000
options.simulator.seed_simulator = 42

print("Configured Options:")
print(f"  optimization_level: {options.optimization_level}")
print(f"  resilience_level: {options.resilience_level}")
print(f"  execution.shots: {options.execution.shots}")
print(f"  simulator.seed_simulator: {options.simulator.seed_simulator}")

# Create a test circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# For local testing with StatevectorSampler
sampler = StatevectorSampler()
job = sampler.run([qc], shots=options.execution.shots)
result = job.result()

print(f"\nBell state results (shots={options.execution.shots}):")
print(f"  Counts: {result[0].data.meas.get_counts()}")

# For real hardware, use:
# from qiskit_ibm_runtime import SamplerV2
# sampler = SamplerV2(backend=backend, options=options)

print("\n✓ Configure before passing to primitive")
print("✓ For real hardware: SamplerV2(backend=backend, options=options)")

### Options Namespace Structure

```
Options()
├── optimization_level (0-3)        [top-level]
├── resilience_level (0-2)          [top-level]
├── execution
│   ├── shots (default 4096)
│   └── init_qubits (default True)
├── simulator
│   └── seed_simulator (optional)
├── transpilation
│   ├── skip_transpilation (False)
│   └── initial_layout ([0,1,2,...])
└── dynamical_decoupling
    ├── enable (True/False)
    └── sequence_type ('XY4', 'XX', etc.)
```

## Part 7: Options Decision Tree (EXAM CRITICAL!)

```
Choose Options based on:

┌─ What's your priority? ──────────────────────┐
│  Speed → optimization_level=1, resilience=0  │
│  Quality → optimization_level=3, resilience=1│
│  Research → optimization_level=3, resilience=2│
└───────────────────────────────────────────────┘

┌─ What hardware? ─────────────────────────────┐
│  Simulator → resilience_level=0              │
│  Real hardware → resilience_level=1          │
│  Noisy hardware → resilience_level=2         │
└───────────────────────────────────────────────┘

┌─ How precise? ───────────────────────────────┐
│  Quick test → shots=1024                     │
│  Standard → shots=4096                       │
│  High precision → shots=8192+                │
└───────────────────────────────────────────────┘
```

### Common Configuration Patterns

```python
# Rapid Prototyping (simulator)
options = Options()
options.optimization_level = 1
options.resilience_level = 0
options.execution.shots = 1024

# Production (real hardware)
options = Options()
options.optimization_level = 3
options.resilience_level = 1
options.execution.shots = 4096
options.dynamical_decoupling.enable = True

# Maximum Quality (research)
options = Options()
options.optimization_level = 3
options.resilience_level = 2
options.execution.shots = 8192
```

**Memory Aid**: "Options = Optimization + Resilience + Shots"

## 📝 Practice Questions

### Question 1: Default Shots

**What is the default value for execution.shots?**

A) 1024  
B) 2048  
C) 4096  
D) 8192

<details>
<summary>Answer</summary>

**C) 4096**

```python
options = Options()
print(options.execution.shots)  # 4096
```

**CRITICAL**: NOT 1024 (old default)!
</details>

---

### Question 2: Optimization Level Default

**What is the default optimization_level in Options()?**

A) 0  
B) 1  
C) 2  
D) 3

<details>
<summary>Answer</summary>

**C) 2**

```python
options = Options()
print(options.optimization_level)  # 2
```

**Note**: Different from transpile() default (1)!
</details>

---

### Question 3: Accessing Shots

**How to access shots configuration?**

A) `options.shots`  
B) `options.execution.shots`  
C) `options.run.shots`  
D) `options.config.shots`

<details>
<summary>Answer</summary>

**B) `options.execution.shots`**

```python
# CORRECT - nested under execution
options.execution.shots = 1000  # ✓

# WRONG - not top-level
options.shots = 1000  # ❌
```
</details>

---

### Question 4: Resilience Level

**What does resilience_level=1 enable?**

A) Circuit optimization  
B) Error mitigation  
C) More shots  
D) Transpilation

<details>
<summary>Answer</summary>

**B) Error mitigation**

Resilience levels:
- 0: No mitigation
- 1: Basic (TREX)
- 2: Advanced (ZNE)

Mitigates readout/gate errors on real hardware.
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **Default Values**
   - `execution.shots = 4096` (NOT 1024!)
   - `optimization_level = 2` (NOT 1 like transpile!)
   - `resilience_level = 0` (no mitigation)

2. **Namespace Structure**
   - Top-level: optimization_level, resilience_level
   - Nested execution: shots, init_qubits
   - Nested simulator: seed_simulator

3. **Resilience**
   - Level 0: None
   - Level 1: Basic (TREX)
   - Level 2: Advanced (ZNE)

### Critical Exam Facts

- ✅ Default shots: **4096** (not 1024)
- ✅ Default optimization_level: **2** (not 1)
- ✅ Access: `options.execution.shots` (nested)
- ✅ Resilience: `options.resilience_level` (top-level)
- ✅ Pass to primitive: `Sampler(backend=backend, options=options)`
- ✅ Can update after creation

### Common Traps

- ❌ `options.shots` → ✅ `options.execution.shots`
- ❌ Default 1024 → ✅ Default 4096
- ❌ Both default 1 → ✅ Options=2, transpile=1

### Mnemonic

🧠 **"4096 Shots, Level 2, Resilience 0-1-2!"**

**Next**: Backend Target (V2 API)!